In [6]:
import time
import random


class CircuitBreaker:
    def __init__(self, falhas_max=3, timeout=5):
        self.falhas_max = falhas_max
        self.timeout = timeout
        self.falhas = 0
        self.estado = "CLOSED"
        self.ultimo_tempo = None

    def chamada(self, func):
        if self.estado == "OPEN":
            if time.time() - self.ultimo_tempo < self.timeout:
                raise Exception("Circuito aberto! Tentando novamente depois...")
            else:
                self.estado = "HALF"

        try:
            resultado = func()
            self.falhas = 0
            self.estado = "CLOSED"
            return resultado
        except Exception as e:
            self.falhas += 1
            if self.falhas >= self.falhas_max:
                self.estado = "OPEN"
                self.ultimo_tempo = time.time()
            raise e


def api_instavel():
    if random.random() < 0.7:
        raise Exception("Falha na API")
    return "OK"


cb = CircuitBreaker()

for _ in range(5):
    try:
        print(cb.chamada(api_instavel))
    except Exception as e:
        print("Erro:", e)
    time.sleep(1)

Erro: Falha na API
Erro: Falha na API
Erro: Falha na API
Erro: Circuito aberto! Tentando novamente depois...
Erro: Circuito aberto! Tentando novamente depois...
